In [1]:
from keras.models import Sequential, load_model
from keras.layers import Dense, Activation, Dropout
from keras_tqdm import TQDMNotebookCallback
import keras.callbacks

from sqlalchemy import create_engine
import json
import pickle

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


In [78]:
# R  W  1111111
# R  P  1111111
# G  W  1111111
# G  P  1111111
# H  W  1111111
# H  P  1111111

# category = 'R'
# category = 'G'
category = 'H'

# bet_type = 'W'
bet_type = 'P'

In [79]:
engine = create_engine('sqlite:///../../data/race.db')
conn = engine.connect()
conn
dfoo = pd.read_sql_table('race', conn)
print('{} races loaded!'.format(len(dfoo)))

3832 races loaded!


In [80]:
dfo = dfoo.loc[dfoo['race_type'].isin([category])]
print('{} {} races!'.format(len(dfo), category))

949 H races!


In [81]:
dfo['results'] = dfo['results_data'].map(json.loads)
dfo['runners'] = dfo['runners_data'].map(json.loads)
dfo.head(3)

/Users/jaco/.pyenv/versions/3.6.1/envs/tabby/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/jaco/.pyenv/versions/3.6.1/envs/tabby/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,id,meeting_name,location,venue_mnemonic,race_type,meeting_date,race_number,race_name,race_start_time,race_status,race_distance,results_data,num_runners,runners_data,quinella,exacta,trifecta,first_four,results,runners
36,37,CHARLTON,VIC,M,H,2017-10-01,1,DNR LOGISTICS 3YO PACE MS,2017-10-01 02:14:00,Paying,2100,"[[1], [5], [3], [4]]",6.0,"[{""runnerName"": ""TORQUE TOTHE STARZ"", ""runnerN...",3.3,15.6,36.1,57.3,"[[1], [5], [3], [4]]","[{'runnerName': 'TORQUE TOTHE STARZ', 'runnerN..."
37,38,CHARLTON,VIC,M,H,2017-10-01,2,AGISTMENT KILMORE PACE MS,2017-10-01 02:48:00,Paying,1609,"[[1], [2], [8], [7]]",8.0,"[{""runnerName"": ""SUBTLE EMBRACE"", ""runnerNumbe...",4.3,10.5,33.8,186.1,"[[1], [2], [8], [7]]","[{'runnerName': 'SUBTLE EMBRACE', 'runnerNumbe..."
38,39,CHARLTON,VIC,M,H,2017-10-01,3,VALE ROD GRANT TROT MS,2017-10-01 03:18:00,Paying,2100,"[[5], [1], [6], [2]]",10.0,"[{""runnerName"": ""SHESASUNDON"", ""runnerNumber"":...",6.5,14.0,91.7,249.9,"[[5], [1], [6], [2]]","[{'runnerName': 'SHESASUNDON', 'runnerNumber':..."


In [82]:
# extract runners from races
data_all = pd.DataFrame()
for ri, race in dfo.iterrows():
    data_all = data_all.append(race['runners'])    
data_all.tail(3)

,P_pred,P_prob,W_pred,W_prob,barrierNumber,claimAmount,finishingPosition,fixedOdds,has_odds,num_runners,...,place_rank,place_scaled,riderDriverName,runnerName,runnerNumber,trainerName,win_odds,win_perc,win_rank,win_scaled
6,0.379844,0.144211,0.078995,0.091082,10,0,4,"{'returnWin': 7.5, 'returnWinOpen': 5, 'return...",True,9,...,3.0,0.169450,DEAN MILLER,LAMPARD,1,R J EDDY,7.5,0.133333,3.0,0.112123
7,0.070943,0.026934,0.008431,0.009720,10,0,0,"{'returnWin': 71, 'returnWinOpen': 51, 'return...",True,9,...,8.0,0.033679,DYLAN VALENTI,SKEWSY NEWSY,2,P J DUGGAN,71.0,0.014085,8.0,0.011844
8,0.248358,0.094291,0.039060,0.045037,10,0,2,"{'returnWin': 18, 'returnWinOpen': 5.5, 'retur...",True,9,...,4.0,0.106321,RYAN BELL,RUM DELIGHT,3,R P BELL,18.0,0.055556,5.0,0.046718


In [83]:
# drop scratched
data = data_all.dropna(subset=['win_odds', 'place_odds'])
data = data[(data.win_odds > 0)]
data = data[(data.place_odds > 0)]
data.describe()

,P_pred,P_prob,W_pred,W_prob,barrierNumber,claimAmount,finishingPosition,num_runners,place_odds,place_perc,place_rank,place_scaled,runnerNumber,win_odds,win_perc,win_rank,win_scaled
count,9.101000e+03,9.101000e+03,9101.000000,9101.000000,9101.000000,9101.0,9101.000000,9101.000000,9101.000000,9101.000000,9101.000000,9101.000000,9101.000000,9101.000000,9101.000000,9101.000000,9101.000000
mean,2.858753e-01,1.042743e-01,0.101399,0.104274,0.764641,0.0,0.988573,10.308647,4.761072,0.373053,5.484013,0.104274,5.850236,23.017442,0.130598,5.480607,0.104274
std,1.999191e-01,7.700736e-02,0.137252,0.140827,3.564197,0.0,1.390581,2.964590,33.481694,0.225733,3.302429,0.066467,3.519691,25.541247,0.149608,3.295315,0.118844
min,2.576906e-17,1.661733e-17,0.000000,0.000000,0.000000,0.0,0.000000,4.000000,1.010000,0.000444,1.000000,0.000174,1.000000,1.010000,0.006623,1.000000,0.004005
25%,1.316881e-01,4.807214e-02,0.017647,0.018332,0.000000,0.0,0.000000,8.000000,1.880000,0.188679,3.000000,0.051952,3.000000,6.000000,0.032258,3.000000,0.025563
50%,2.340566e-01,8.389811e-02,0.047719,0.048460,0.000000,0.0,0.000000,10.000000,3.150000,0.317460,5.000000,0.087547,5.000000,14.000000,0.071429,5.000000,0.057942
75%,4.076438e-01,1.467589e-01,0.118177,0.120320,0.000000,0.0,2.000000,12.000000,5.300000,0.531915,8.000000,0.147838,8.000000,31.000000,0.166667,8.000000,0.138751
max,1.000000e+00,7.790164e-01,1.000000,0.964392,80.000000,0.0,4.000000,20.000000,2250.000000,0.990099,20.000000,0.421386,20.000000,151.000000,0.990099,20.000000,0.807534


In [84]:
# get label data
Y = data['finishingPosition']

if bet_type == 'W':
    Y = (Y == 1)
elif bet_type == 'P':
    Y = (Y == 1) | (Y == 2) | ((Y == 3) & (data['num_runners'] >= 8))

#print(data['finishingPosition'].head(10))
Y = Y.astype(int)
# Y.head(10)
Y.describe()

count    9101.000000
mean        0.295792
std         0.456423
min         0.000000
25%         0.000000
50%         0.000000
75%         1.000000
max         1.000000
dtype: float64

In [85]:
xp = data['win_perc']
xs = data['win_scaled']
xr = data['win_rank']
xn = data['num_runners']
X = pd.concat([xp, xs, xr, xn], axis=1)
X.describe()

,win_perc,win_scaled,win_rank,num_runners
count,9101.000000,9101.000000,9101.000000,9101.000000
mean,0.130598,0.104274,5.480607,10.308647
std,0.149608,0.118844,3.295315,2.964590
min,0.006623,0.004005,1.000000,4.000000
25%,0.032258,0.025563,3.000000,8.000000
50%,0.071429,0.057942,5.000000,10.000000
75%,0.166667,0.138751,8.000000,12.000000
max,0.990099,0.807534,20.000000,20.000000


In [87]:
# For a single-input model with 2 classes (binary classification):

n = len(X.columns)
print('input dimension = {}'.format(n))

epochs = 500
print('epochs = {}'.format(epochs))

layer_1 = 30
layer_2 = 30

tag = '{}{}x{}{}'.format(category, layer_1, layer_2, bet_type)
print('tag = {}'.format(tag))
file_name = '/Users/jaco/code/tabby/each_way/v1/models/{}.h5'.format(tag)

try:
    model = load_model(file_name)
    print('model loaded')
except OSError:
    model = Sequential()
    model.add(Dense(layer_1, activation='relu', input_dim=n))
    model.add(Dense(layer_2, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    print('model created')

input dimension = 4
epochs = 500
tag = H30x30P
model loaded


In [88]:
# Train the model, iterating on the data in batches of 32 samples
tqdm = TQDMNotebookCallback()

tbCallBack = keras.callbacks.TensorBoard(
    log_dir='/Users/jaco/code/tabby/each_way/v1/summary/{}'.format(tag), 
    histogram_freq=0,
    write_graph=True,
    write_images=True)

model.fit(
    X.as_matrix(), 
    Y.as_matrix(),
    validation_split=0.2,
    shuffle=True,
    epochs=epochs,
    batch_size=32,
    verbose=0,
    callbacks=[tqdm, tbCallBack])

# creates a HDF5 file 'my_model.h5'
model.save(file_name)